In [ ]:
def get_conversations_html(name):
    return html.Div(
        children=[
            dbc.Select(
                id="llm-type" + name,
                value='gemini-1.5-pro-text',
                options=[
                    {'label': 'Gemini Flash', 'value': 'gemini-1.5-flash-text'},
                    {'label': 'Gemini Pro', 'value': 'gemini-1.5-pro-text'}
                ],
            ),
            html.Div(
                id="display-conversation" + name,
                style={
                    "overflow-y": "auto",  # Allow vertical scrolling
                    "height": "calc(85vh - 100px)",  # Fixed height minus header/input
                    "flex": "1"
                }
            ),
        ],
        style={
            "display": "flex",
            "flexDirection": "column",
            "height": "85vh",
            "width": "400px",  # Fixed width for the chat area
            "backgroundColor": "#eeeeee",
            "position": "relative"  # Keep it in the document flow
        }
    )

def setup_layout(name):
    return html.Div(
        style={
            'width': '100%',
            'height': '100vh',
            'margin': 0,
            'padding': 0,
            'overflow': 'hidden'
        },
        children=[
            uitk.HeaderAllIn(
                menu_items=[{
                    'type': 'outlined',
                    'id': 'unique-id' + name,
                }],
                brand={'appName': f'{name} BYOD', 'envBadge': {'name': f'{ENV}'}},
                search_expand_mode='none'
            ),
            html.Div(id='page-load-div' + name, style={'display': 'none'}),
            html.Div(
                children=[
                    uitk.Row(
                        children=[
                            uitk.Col(
                                children=[
                                    html.Div(
                                        children=[
                                            dcc.Upload(
                                                id='upload-data' + name,
                                                children=html.Div(['Drag and Drop or ', html.A('Select Files (.txt, .pdf)')]),
                                                style={
                                                    'width': '100%',
                                                    'height': '60px',
                                                    'lineHeight': '60px',
                                                    'borderWidth': '1px',
                                                    'borderStyle': 'dashed',
                                                    'borderRadius': '5px',
                                                    'textAlign': 'center',
                                                    'margin': '10px'
                                                },
                                                multiple=True
                                            ),
                                            uitk.DynamicTabs(
                                                id='file_tabs' + name,
                                                active_tab='tab1' + name,
                                                tabs=[{}]
                                            ),
                                        ],
                                        style={
                                            'flex': '1',
                                            'overflow': 'hidden'
                                        }
                                    ),
                                ],
                                style={
                                    'flex': '1',
                                    'minWidth': '0'  # Important for flex layout
                                }
                            ),
                            uitk.Col(
                                children=[
                                    dbc.Container(
                                        children=[
                                            dcc.Store(id='store-conversation' + name, data=""),
                                            get_conversations_html(name),
                                            get_controls_html(name),
                                            dbc.Spinner(html.Div(id='loading-component' + name))
                                        ]
                                    )
                                ],
                                style={
                                    'width': '400px',  # Fixed width
                                    'flex': 'none',    # Prevent flexing
                                    'paddingLeft': '10px'
                                }
                            ),
                        ],
                        style={
                            'display': 'flex',
                            'height': 'calc(100vh - 64px)',  # Full height minus header
                            'overflow': 'hidden'
                        }
                    ),
                    html.Div(id='hidden-div' + name, style={'display': 'none'}),
                    uitk.Spinner(
                        html.Div(
                            id='storing-file' + name,
                            children=[dcc.Store(id='uploaded-file' + name)],
                            style={'minHeight': '50px'}
                        )
                    )
                ],
                style={
                    'display': 'flex',
                    'flexDirection': 'column',
                    'height': 'calc(100vh - 64px)'  # Full height minus header
                }
            )
        ]
    )